In [35]:
!rm -rf /tmp/wordcount
!mkdir -p /tmp/wordcount/input
%cd /tmp/wordcount
!ls

sh: 0: getcwd() failed: No such file or directory
/tmp/wordcount
input


In [36]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [37]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [38]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing input/text2.txt


In [39]:
!which python3

/usr/bin/python3


In [40]:
%%writefile mapper.py
#! /usr/bin/python3

import sys

if __name__ == "__main__":

    for line in sys.stdin:
        for word in line.split():
            sys.stdout.write("{}\t1\n".format(word))

Writing mapper.py


In [41]:
%%writefile reducer.py
#! /usr/bin/python3

import sys

if __name__ == '__main__':

    curkey = None
    total = 0

    for line in sys.stdin:

        key, val = line.split("\t")
        val = int(val)

        if key == curkey:
            total += val
        else:
            if curkey is not None:
                sys.stdout.write("{}\t{}\n".format(curkey, total))

            curkey = key
            total = val

    sys.stdout.write("{}\t{}\n".format(curkey, total))

Writing reducer.py


In [42]:
%cd /tmp/wordcount
!chmod +x mapper.py reducer.py

/tmp/wordcount


In [43]:
!ls  -1

input
mapper.py
reducer.py


In [44]:
!cat input/text*.txt | python3 mapper.py | sort | python3 reducer.py | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [45]:
!ls -1 input/

text0.txt
text1.txt
text2.txt


In [46]:
!hadoop fs -mkdir input
!hadoop fs -copyFromLocal input/* input
!hadoop fs -ls input/*
#
# Se puede usar la webUI para comprobar el sistema de ficheros, pestaña "Utilities - Browse the file system"
#

-rw-r--r--   1 root supergroup       1082 2023-10-17 08:33 input/text0.txt
-rw-r--r--   1 root supergroup        349 2023-10-17 08:33 input/text1.txt
-rw-r--r--   1 root supergroup        435 2023-10-17 08:33 input/text2.txt


In [47]:
%%writefile app.sh
#
# Se ejecuta en Hadoop.
#   -files: archivos a copiar al hdfs
#   -input: archivo de entrada
#   -output: directorio de salida
#   -file: archivos a copiar de la maquina local al hdfs
#   -mapper: programa que ejecuta el map
#   -reducer: programa que ejecuta la reducción
#
hadoop fs -rm -r output

hadoop jar \
    $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
    -files mapper.py,reducer.py  \
    -input input  \
    -output output \
    -mapper mapper.py \
    -reducer reducer.py

Writing app.sh


In [48]:
#
# Lanzamos MapReduce
#
!bash app.sh
#
# Se pueden ver los procesos en la webUI de YARN
#

rm: `output': No such file or directory
packageJobJar: [/tmp/hadoop-unjar4041784982998829825/] [] /tmp/streamjob8337579124133857454.jar tmpDir=null


In [49]:
#
# Contenido del directorio con los resultados de la ejecución
#
!hadoop fs -ls output

Found 2 items
-rw-r--r--   1 root supergroup          0 2023-10-17 08:34 output/_SUCCESS
-rw-r--r--   1 root supergroup       1649 2023-10-17 08:33 output/part-00000


In [50]:
#
# Se visualiza el archivo con los resultados de la ejecución
#
!hadoop fs -cat output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [51]:
%%writefile app_comb.sh
hadoop fs -rm -r output
hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
   -files mapper.py,reducer.py \
   -input input \
   -output output  \
   -mapper mapper.py \
   -reducer reducer.py \
   -combiner reducer.py

Writing app_comb.sh


In [52]:
#
# A diferencia del script anterior, en este caso se usa también una función "combiner" 
# cuyo código es el mismo de "reducer"
#
!bash app_comb.sh

Deleted output
packageJobJar: [/tmp/hadoop-unjar7209985952556640210/] [] /tmp/streamjob2902073394292491345.jar tmpDir=null


In [53]:
!hadoop fs -cat output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [54]:
!hadoop fs -rm -r -f -skipTrash input
!hadoop fs -rm -r -f -skipTrash output

Deleted input
Deleted output
